In [1]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from gensim.models import KeyedVectors
# from albert import *
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

/home/salsabila.pranida/.conda/envs/ai_aes/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
# Constants
BASE_DIR = '../'  # Navigate one level up to access directories outside of albert_ira
DATASET_DIR = os.path.join(BASE_DIR, 'dataset')
SAVE_DIR = os.path.join(BASE_DIR, 'result')
MODEL_NAME = "albert-base-v2"
GLOVE_PATH = os.path.join(BASE_DIR, 'word_embeddings/glove.6B.300d.txt')
FASTTEXT_PATH = os.path.join(BASE_DIR, 'word_embeddings/wiki.en.vec')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
albert_model = AutoModel.from_pretrained(MODEL_NAME).to(device)

directories = [BASE_DIR, DATASET_DIR, SAVE_DIR, os.path.dirname(GLOVE_PATH), os.path.dirname(FASTTEXT_PATH)]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory created: {directory}")
    else:
        print(f"Directory already exists: {directory}")

2024-11-15 08:14:49.627401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731644090.327749 3816963 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731644090.531084 3816963 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 08:14:52.875558: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Directory already exists: ../
Directory already exists: ../dataset
Directory already exists: ../result
Directory already exists: ../word_embeddings
Directory already exists: ../word_embeddings


In [4]:
def load_glove_model(glove_file_path):
    """Load GloVe embeddings from file into a dictionary."""
    embedding_dict = {}
    with open(glove_file_path, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = torch.tensor(np.asarray(values[1:], dtype='float32'))
            embedding_dict[word] = vector.to(device)  # Move to device if necessary
    return embedding_dict

def load_fasttext_model(fasttext_file_path):
    """Load FastText embeddings from file into a dictionary."""
    model = KeyedVectors.load_word2vec_format(fasttext_file_path, binary=False)
    return {word: torch.tensor(model[word]).to(device) for word in model.index_to_key}

# Load embeddings
glove_model = load_glove_model(GLOVE_PATH)
fasttext_model = load_fasttext_model(FASTTEXT_PATH)

In [5]:
# Load and preprocess the dataset
df = pd.read_csv('processed_essay_dataset.csv', sep=',', encoding='ISO-8859-1')
df = df.dropna(subset=['normalized_score','content', 'prompt_adherence', 'language', 'narrativity'])  # Ensure all required columns are present
df.fillna(0, inplace=True)

q1, q3 = df['normalized_score'].quantile([0.25, 0.75])
df['quality_label'] = pd.cut(df['normalized_score'], bins=[-1, q1, q3, 100], labels=[0, 1, 2]).astype(int)
df['quality_label'] = df['quality_label'].map({0: 0, 1: 1, 2: 2})

# Map essay types as before and filter
df['essay_type'] = df['essay_type'].map({'argumentative': 0, 'dependent': 1, 'narrative': 2})
df = df[df['essay_type'].isin([0, 1])]

df.head()


,essay_id,essay_set,essay,essay_type,domain1_score,content,organization,word_choice,sentence_fluency,conventions,language,prompt_adherence,narrativity,style,voice,normalized_score,quality_label
3573,5978,3,The features of the setting affect the cyclist...,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,33.333333,0
3574,5979,3,The features of the setting affected the cycli...,1,2.0,3.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,66.666667,1
3575,5980,3,Everyone travels to unfamiliar places. Sometim...,1,1.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,33.333333,0
3576,5981,3,I believe the features of the cyclist affected...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,33.333333,0
3577,5982,3,The setting effects the cyclist because of the...,1,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,66.666667,1


In [ ]:
# Check each feature column and display its statistics if it exists
feature_descriptions = {}
for column in feature_columns:
    if column in df.columns:
        feature_descriptions[column] = df[column].describe()
    else:
        feature_descriptions[column] = f"Column '{column}' does not exist in the dataset."

# Display the results
feature_descriptions

In [ ]:
class MultiTaskModel(nn.Module):
    def __init__(self, input_shape, num_classes=6):
        super(MultiTaskModel, self).__init__()
        self.fc1 = nn.Linear(input_shape, 256)
        self.bn1 = nn.BatchNorm1d(256)  # Batch normalization
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.5)

        self.content_head = nn.Linear(128, num_classes)
        self.language_head = nn.Linear(128, num_classes)
        self.prompt_adherence_head = nn.Linear(128, num_classes)
        self.narrativity_head = nn.Linear(128, num_classes)

    def forward(self, x):
        # Check if batch size is 1 to skip batch normalization
        if x.size(0) > 1:
            x = torch.relu(self.bn1(self.fc1(x)))
        else:
            x = torch.relu(self.fc1(x))
        x = self.dropout1(x)

        if x.size(0) > 1:
            x = torch.relu(self.bn2(self.fc2(x)))
        else:
            x = torch.relu(self.fc2(x))
        x = self.dropout2(x)

        language_output = self.language_head(x)
        prompt_adherence_output = self.prompt_adherence_head(x)
        narrativity_output = self.narrativity_head(x)

        return language_output, prompt_adherence_output, narrativity_output

    def compute_uncertainty_loss(self, loss_content, loss_language, loss_prompt_adherence, loss_narrativity):
        """Compute the weighted uncertainty loss for each feature."""
        language_precision = torch.exp(-self.task_uncertainty[1])
        prompt_adherence_precision = torch.exp(-self.task_uncertainty[2])
        narrativity_precision = torch.exp(-self.task_uncertainty[3])

        # Weighted loss calculation
        loss = (language_precision * loss_language + self.task_uncertainty[1]) + \
               (prompt_adherence_precision * loss_prompt_adherence + self.task_uncertainty[2]) + \
                    (narrativity_precision * loss_narrativity + self.task_uncertainty[3]) 
        
        return loss

    def compute_loss(self, pred_language, pred_prompt_adherence, pred_narrativity,
                    y_content, y_language, y_prompt_adherence, y_narrativity) :

        # mse_loss = nn.MSELoss()(pred_score, y_score)
        # cross_entropy_loss_quality = nn.CrossEntropyLoss()(pred_quality, y_quality)
        # cross_entropy_loss_essay_type = nn.CrossEntropyLoss()(pred_essay_type, y_essay_type)

        # MSE loss for the additional attributes
        criterion = nn.CrossEntropyLoss()
        mse_loss_language = criterion(pred_language, y_language)
        mse_loss_prompt_adherence = criterion(pred_prompt_adherence, y_prompt_adherence)
        mse_loss_narrativity = criterion(pred_narrativity, y_narrativity)

        # Sum all the losses for total loss
        total_loss = mse_loss_language + mse_loss_prompt_adherence + mse_loss_narrativity         
        return total_loss


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        log_probs = F.log_softmax(pred, dim=-1)
        nll_loss = -log_probs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -log_probs.mean(dim=-1)
        return (1 - self.smoothing) * nll_loss + self.smoothing * smooth_loss

In [18]:
def get_albert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=256).to(device)
    with torch.no_grad():
        outputs = albert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

def get_word_embedding(text, embedding_dict):
    words = text.lower().split()
    vectors = [embedding_dict[word] for word in words if word in embedding_dict]
    if vectors:
        return torch.mean(torch.stack(vectors), dim=0).cpu().numpy()
    return np.zeros(300)

# Create attention-based embedding fusion
def create_attention_based_embedding(albert_emb, additional_emb):
    if albert_emb.shape != additional_emb.shape:
        additional_emb = torch.nn.Linear(additional_emb.shape[0], albert_emb.shape[0]).to(albert_emb.device)(additional_emb)
    combined_emb = torch.cat([albert_emb.unsqueeze(0), additional_emb.unsqueeze(0)], dim=0)
    attention_weights = torch.nn.Parameter(torch.tensor([0.5, 0.5], device=albert_emb.device), requires_grad=True)
    attention_scores = F.softmax(attention_weights, dim=0)
    fused_embedding = attention_scores[0] * albert_emb + attention_scores[1] * additional_emb
    return fused_embedding

def create_combined_embedding(text, embedding_type=None, _glove_model=None, _fasttext_model=None):
    albert_emb = get_albert_embedding(text).flatten()

    if embedding_type == "glove":
        additional_emb = get_word_embedding(text, _glove_model)
    elif embedding_type == "fasttext":
        additional_emb = get_word_embedding(text, _fasttext_model)
    else:
        additional_emb = np.array([])

    albert_emb_tensor = torch.tensor(albert_emb, dtype=torch.float32).to(device)

    if additional_emb.size != 0:
        additional_emb_tensor = torch.tensor(additional_emb, dtype=torch.float32).to(device)
        
        # Ensure both embeddings have the same size by truncating or padding
        if additional_emb_tensor.size(0) > albert_emb_tensor.size(0):
            additional_emb_tensor = additional_emb_tensor[:albert_emb_tensor.size(0)]  # Truncate
        elif additional_emb_tensor.size(0) < albert_emb_tensor.size(0):
            padding_size = albert_emb_tensor.size(0) - additional_emb_tensor.size(0)
            additional_emb_tensor = F.pad(additional_emb_tensor, (0, padding_size))  # Pad with zeros

        combined_emb = torch.cat([albert_emb_tensor, additional_emb_tensor], dim=0)
    else:
        combined_emb = albert_emb_tensor

    return combined_emb.cpu().numpy(), combined_emb.size(0)


In [ ]:
def train_and_save_model(X_train_tensor, y_train_language_tensor, 
                         y_train_prompt_adherence_tensor, y_train_narrativity_tensor, 
                         input_shape, save_dir, embedding_type=None, epochs=10, 
                         batch_size=8, learning_rate=1e-4):

    model = MultiTaskModel(input_shape).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_loader = DataLoader(TensorDataset(
        X_train_tensor, y_train_language_tensor, 
        y_train_prompt_adherence_tensor, y_train_narrativity_tensor
    ), batch_size=max(2, batch_size), shuffle=True)  # Ensure batch size is at least 2

    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for X_batch, y_language_batch, y_prompt_adherence_batch, y_narrativity_batch in train_loader:
            # Move data to device
            X_batch = X_batch.to(device)
            y_language_batch = y_language_batch.to(device)
            y_prompt_adherence_batch = y_prompt_adherence_batch.to(device)
            y_narrativity_batch = y_narrativity_batch.to(device)

            optimizer.zero_grad()
            
            # Get model predictions
            pred_language, pred_prompt_adherence, pred_narrativity = model(X_batch)

            # Compute the losses
            criterion = nn.CrossEntropyLoss()
            mse_loss_language = criterion(pred_language, y_language_batch)
            mse_loss_prompt_adherence = criterion(pred_prompt_adherence, y_prompt_adherence_batch)
            mse_loss_narrativity = criterion(pred_narrativity, y_narrativity_batch)

            # Total loss
            total_loss = mse_loss_language + mse_loss_prompt_adherence + mse_loss_narrativity

            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            epoch_loss += total_loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Total Epoch Loss: {epoch_loss / len(train_loader):.4f}")

    # Save the model
    model_filename = f"albert7_model_{embedding_type or 'albert'}.pth"
    embedding_size_filename = f"albert7_embedding_size_{embedding_type or 'albert'}.npy"
    torch.save(model.state_dict(), os.path.join(save_dir, model_filename))
    np.save(os.path.join(save_dir, embedding_size_filename), input_shape)
    return os.path.join(save_dir, model_filename)

In [ ]:
def evaluate_model(model_path, X_test_tensor, y_test_language_tensor, 
                   y_test_prompt_adherence_tensor, y_test_narrativity_tensor):

    model = MultiTaskModel(X_test_tensor.shape[1]).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        # Move data to device
        X_test_tensor = X_test_tensor.to(device)
        y_test_language_tensor = y_test_language_tensor.to(device)
        y_test_prompt_adherence_tensor = y_test_prompt_adherence_tensor.to(device)
        y_test_narrativity_tensor = y_test_narrativity_tensor.to(device)

        # Get predictions
        pred_language, pred_prompt_adherence, pred_narrativity = model(X_test_tensor)

        # Compute Kappa scores
        kappa_language = cohen_kappa_score(y_test_language_tensor.cpu().numpy(), 
                                           np.round(pred_language.cpu().numpy().squeeze()).astype(int), weights='quadratic')
        kappa_prompt_adherence = cohen_kappa_score(y_test_prompt_adherence_tensor.cpu().numpy(), 
                                                   np.round(pred_prompt_adherence.cpu().numpy().squeeze()).astype(int), weights='quadratic')
        kappa_narrativity = cohen_kappa_score(y_test_narrativity_tensor.cpu().numpy(), 
                                              np.round(pred_narrativity.cpu().numpy().squeeze()).astype(int), weights='quadratic')

        print(f"Kappa for Language: {kappa_language:.5f}")
        print(f"Kappa for Prompt Adherence: {kappa_prompt_adherence:.5f}")
        print(f"Kappa for Narrativity: {kappa_narrativity:.5f}")

        return kappa_language, kappa_prompt_adherence, kappa_narrativity

In [ ]:
for embedding_type in [None, "glove", "fasttext"]:
    print(f"Training for embedding type: {embedding_type or 'default'}")

    # Prepare embeddings and split data
    embeddings_and_sizes = df['essay'].apply(lambda x: create_combined_embedding(x, embedding_type, glove_model, fasttext_model))
    df['embeddings'], _ = zip(*embeddings_and_sizes)

    X_train, X_test,y_train_language, y_test_language, \
    y_train_prompt_adherence, y_test_prompt_adherence, y_train_narrativity, y_test_narrativity = train_test_split(
        np.stack(df['embeddings'].values),
        df['language'].values,
        df['prompt_adherence'].values,
        df['narrativity'].values,
        test_size=0.2,
        random_state=42
    )

    # Convert data to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_language_tensor = torch.tensor(y_train_language, dtype=torch.float32)
    y_train_prompt_adherence_tensor = torch.tensor(y_train_prompt_adherence, dtype=torch.float32)
    y_train_narrativity_tensor = torch.tensor(y_train_narrativity, dtype=torch.float32)

    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_language_tensor = torch.tensor(y_test_language, dtype=torch.float32)
    y_test_prompt_adherence_tensor = torch.tensor(y_test_prompt_adherence, dtype=torch.float32)
    y_test_narrativity_tensor = torch.tensor(y_test_narrativity, dtype=torch.float32)

    # Train and save model
    model_path = train_and_save_model(
        X_train_tensor, y_train_language_tensor, 
        y_train_prompt_adherence_tensor, y_train_narrativity_tensor, 
        input_shape=X_train_tensor.shape[1], save_dir=SAVE_DIR, 
        embedding_type=embedding_type, epochs=10, batch_size=8, learning_rate=1e-3
    )

    # Evaluate the model
    kappa_content, kappa_language, kappa_prompt_adherence, kappa_narrativity = evaluate_model(
        model_path, X_test_tensor, y_test_language_tensor, 
        y_test_prompt_adherence_tensor, y_test_narrativity_tensor
    )

Training for embedding type: default


/home/salsabila.pranida/.conda/envs/ai_aes/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10, Total Epoch Loss: 4.8165
Epoch 2/10, Total Epoch Loss: 4.3471
Epoch 3/10, Total Epoch Loss: 4.3157
Epoch 4/10, Total Epoch Loss: 4.2699
Epoch 5/10, Total Epoch Loss: 4.2382
Epoch 6/10, Total Epoch Loss: 4.2208
Epoch 7/10, Total Epoch Loss: 4.2318
Epoch 8/10, Total Epoch Loss: 4.1911
Epoch 9/10, Total Epoch Loss: 4.1974
Epoch 10/10, Total Epoch Loss: 4.1994
Kappa for Content: -0.06453
Kappa for Language: -0.06604
Kappa for Prompt Adherence: -0.07050
Kappa for Narrativity: -0.08467
Training for embedding type: glove


/tmp/slurm-salsabila.pranida-51449/ipykernel_3816963/761690038.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=

Epoch 1/10, Total Epoch Loss: 4.8646
Epoch 2/10, Total Epoch Loss: 4.3209
Epoch 3/10, Total Epoch Loss: 4.2696
Epoch 4/10, Total Epoch Loss: 4.2514
Epoch 5/10, Total Epoch Loss: 4.2351
Epoch 6/10, Total Epoch Loss: 4.2591
Epoch 7/10, Total Epoch Loss: 4.2729
Epoch 8/10, Total Epoch Loss: 4.3825
Epoch 9/10, Total Epoch Loss: 4.1921
Epoch 10/10, Total Epoch Loss: 4.2183
Kappa for Content: -0.00161
Kappa for Language: 0.10471
Kappa for Prompt Adherence: 0.00977
Kappa for Narrativity: -0.01247
Training for embedding type: fasttext


/tmp/slurm-salsabila.pranida-51449/ipykernel_3816963/761690038.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=

Epoch 1/10, Total Epoch Loss: 4.8495
Epoch 2/10, Total Epoch Loss: 4.3377
Epoch 3/10, Total Epoch Loss: 4.3121
Epoch 4/10, Total Epoch Loss: 4.2546
Epoch 5/10, Total Epoch Loss: 4.2368
Epoch 6/10, Total Epoch Loss: 4.2267
Epoch 7/10, Total Epoch Loss: 4.2010
Epoch 8/10, Total Epoch Loss: 4.1896
Epoch 9/10, Total Epoch Loss: 4.2826
Epoch 10/10, Total Epoch Loss: 4.1623
Kappa for Content: 0.16897
Kappa for Language: 0.17681
Kappa for Prompt Adherence: 0.08334
Kappa for Narrativity: 0.15129


/tmp/slurm-salsabila.pranida-51449/ipykernel_3816963/761690038.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=

In [23]:
content = """
    In “Let there be dark,” Paul Bogard talks about the importance of darkness.
Darkness is essential to humans. Bogard states, “Our bodies need darkness to produce the hormone melatonin, which keeps certain cancers from developing, and our bodies need darkness for sleep, sleep. Sleep disorders have been linked to diabetes, obesity, cardiovascular disease and depression and recent research suggests are main cause of “short sleep” is “long light.” Whether we work at night or simply take our tablets, notebooks and smartphones to bed, there isn’t a place for this much artificial light in our lives.” (Bogard 2). Here, Bogard talks about the importance of darkness to humans. Humans need darkness to sleep in order to be healthy.
Animals also need darkness. Bogard states, “The rest of the world depends on darkness as well, including nocturnal and crepuscular species of birds, insects, mammals, fish and reptiles. Some examples are well known—the 400 species of birds that migrate at night in North America, the sea turtles that come ashore to lay their eggs—and some are not, such as the bats that save American farmers billions in pest control and the moths that pollinate 80% of the world’s flora. Ecological light pollution is like the bulldozer of the night, wrecking habitat and disrupting ecosystems several billion years in the making. Simply put, without darkness, Earth’s ecology would collapse...” (Bogard 2). Here Bogard explains that animals, too, need darkness to survive.
""" 

In [28]:
def testContent(content, embedding_type=None, SAVE_DIR=None, glove_model=None, fasttext_model=None):
    # Generate the combined embedding
    embedding, actual_embedding_size = create_combined_embedding(
        content,
        embedding_type=embedding_type,
        _glove_model=glove_model if embedding_type == "glove" else None,
        _fasttext_model=fasttext_model if embedding_type == "fasttext" else None
    )

    embedding_tensor = torch.tensor(embedding, dtype=torch.float32).to(device).unsqueeze(0)

    # Load model files
    embedding_size_filename = f"albert7_embedding_size_{embedding_type or 'albert'}.npy"
    model_filename = f"albert7_model_{embedding_type or 'albert'}.pth"

    # Load the expected embedding size and model
    embedding_size_path = os.path.join(SAVE_DIR, embedding_size_filename)
    expected_embedding_size = int(np.load(embedding_size_path))

    # Initialize model and load weights
    model = MultiTaskModel(expected_embedding_size).to(device)
    model_path = os.path.join(SAVE_DIR, model_filename)
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()

    # Adjust embedding size if necessary
    embedding_resized = embedding_tensor[:, :expected_embedding_size]

    # Make predictions
    with torch.no_grad():
        pred_content, pred_language, pred_prompt_adherence, pred_narrativity = model(embedding_resized)
        content_score = pred_content.cpu().item()
        language_score = pred_language.cpu().item()
        prompt_adherence_score = pred_prompt_adherence.cpu().item()
        narrativity_score = pred_narrativity.cpu().item()

    return content_score, language_score, prompt_adherence_score, narrativity_score

In [29]:
# Initialize an empty dictionary to store results for each embedding type
results = {}

# Define the embedding types
embedding_types = [None, "glove", "fasttext"]

for embedding_type in embedding_types:
    # Define embedding type name for readability
    if embedding_type is None:
        embedding_type_name = "ALBERT"
    elif embedding_type == "glove":
        embedding_type_name = "ALBERT + GloVe"
    elif embedding_type == "fasttext":
        embedding_type_name = "ALBERT + FastText"

    # Generate the scores for the given content and embedding type
    content_score, language_score, prompt_adherence_score, narrativity_score = testContent(
        content=content,
        embedding_type=embedding_type,
        SAVE_DIR=SAVE_DIR,
        glove_model=glove_model,
        fasttext_model=fasttext_model
    )
    print(f"Embedding Type: {embedding_type_name}")
    print(f"Returned Values: {content_score}, {language_score}, {prompt_adherence_score}, {narrativity_score}")


    try:
        results[embedding_type_name] = {
            "Content Score": float(content_score),
            "Language Score": float(language_score),
            "Prompt Adherence Score": float(prompt_adherence_score),
            "Narrativity Score": float(narrativity_score),
        }
    except ValueError:
        print(f"Error: Unable to convert one or more values to float for embedding type: {embedding_type_name}")
        print(f"Values: {content_score}, {language_score}, {prompt_adherence_score}, {narrativity_score}")

# Display the results for each embedding type
for embedding_name, result in results.items():
    print(f"Sample Essay Scores for {embedding_name}:")
    print(f"  Content Score: {result['Content Score']:.2f}")
    print(f"  Language Score: {result['Language Score']:.2f}")
    print(f"  Prompt Adherence Score: {result['Prompt Adherence Score']:.2f}")
    print(f"  Narrativity Score: {result['Narrativity Score']:.2f}")

Embedding Type: ALBERT
Returned Values: 3.213857650756836, 3.540748119354248, 3.555917978286743, 3.510068893432617
Embedding Type: ALBERT + GloVe
Returned Values: 2.8946549892425537, 3.3535208702087402, 3.113101005554199, 2.919105291366577
Embedding Type: ALBERT + FastText
Returned Values: 3.9327754974365234, 4.1273627281188965, 4.03092622756958, 4.10561466217041
Sample Essay Scores for ALBERT:
  Content Score: 3.21
  Language Score: 3.54
  Prompt Adherence Score: 3.56
  Narrativity Score: 3.51
Sample Essay Scores for ALBERT + GloVe:
  Content Score: 2.89
  Language Score: 3.35
  Prompt Adherence Score: 3.11
  Narrativity Score: 2.92
Sample Essay Scores for ALBERT + FastText:
  Content Score: 3.93
  Language Score: 4.13
  Prompt Adherence Score: 4.03
  Narrativity Score: 4.11


/tmp/slurm-salsabila.pranida-51449/ipykernel_3816963/2665487501.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)